## `PioModelInitializer` Class
Must implement the `initialize_model()` method.

In [ ]:
class PioModelInitializer(object):
    def __init__(self, 
                 *args,
                 **kwargs):        

        pass

    
    def initialize_model(self,
                        *args,
                        **kwargs):

        return

## `PioRequestTransformer` Class
Must implement the `transform_request()` method.

In [ ]:
class PioRequestTransformer(object):
    def __init__(self, 
                 *args,
                 **kwargs):        
        pass
    
    
    def transform_request(self,
                          request,
                          *args,
                          **kwargs):
        return request

## `PioResponseTransformer` Class
Must implement the `transform_response()` method.

In [ ]:
class PioResponseTransformer(object):
    def __init__(self, 
                 *args,
                 **kwargs):        
        pass
    
    
    def transform_response(self,
                           response,
                           *args,
                           **kwargs):
        return response

## `PioModel` Class
Must implement the `predict()` method.

In [ ]:
class PioModel(object):

    def __init__(self, 
                 request_transformer, 
                 response_transformer,
                 model_initializer,
                 *args,
                 **kwargs):

        self.request_transformer = request_transformer
        self.response_transformer = response_transformer

        self.model_initializer = model_initializer
        self.model = self.model_initializer.initialize_model(args,
                                                             kwargs)

        
    def predict(self, 
                request,
                *args,
                **kwargs):

        return

## `MyResponseTransformer` Class
Extends `PioResponseTransformer` class.  Must implement the `transform_response()` method.

### Convert `json` -> `dict`

In [ ]:
class MyRequestTransformer(PioRequestTransformer):        
    def __init__(self, 
                 *args,
                 **kwargs):
        
        PioRequestTransformer.__init__(self, 
                                       args, 
                                       kwargs)

        
    def transform_request(self,
                          request,
                          *args,
                          **kwargs):
        
        request_str = request.decode('utf-8')
        request_str = request_str.strip().replace('\n', ',')
        request_dict = json.loads(request_str)
        return request_dict

## Create Response Transformer
Must have a `transform_response()` method.

### Convert `dict` -> `json`

In [ ]:
class MyResponseTransformer(PioResponseTransformer):

    def transform_response(self,
                           response):
        response_json = json.dumps(response)
        return response_json

## Train My Model

In [ ]:
## No training is needed.  "Learned Variables" are hard-coded in this example.

## Create Model Initializer
Must implement the `initialize_model()` method

In [ ]:
class MyModelInitializer(PioModelInitializer): 

    def __init__(self,
                 *args,
                 **kwargs):

        PioModelInitializer.__init__(self, 
                                     args, 
                                     kwargs)

        
    def initialize_model(self,                        
                         *args,
                         **kwargs):

        PioModelInitializer.initialize_model(self, 
                                             args, 
                                             kwargs)

        model = {'cat_mean':0.1,
                 'cat_stdv':0.20,
                 'dog_mean':0.3,
                 'dog_stdv':0.40}
        
        return model            

## `MyModel` Class
Extends `PioModel` class.  Must implement `predict()` method.

In [ ]:
class MyModel(PioModel):

    def __init__(self, 
                 request_transformer, 
                 response_transformer,
                 model_initializer,
                 *args,
                 **kwargs):

        PioModel.__init__(self,
                          request_transformer,
                          response_transformer,
                          model_initializer,
                          args,
                          kwargs)

    
    def predict(self, 
                request):
        transformed_request = self.request_transformer.transform_request(request)

        cat_affinity_score = sum([ d['weight'] * d['user_score'] for d in transformed_request if 'cat' in d['tags'] ])
        dog_affinity_score = sum([ d['weight'] * d['user_score'] for d in transformed_request if 'dog' in d['tags'] ])

        # create normalized z score for compare/classify
        cat_zscore = (cat_affinity_score - self.model['cat_mean'])/self.model['cat_stdv']
        dog_zscore = (dog_affinity_score - self.model['dog_mean'])/self.model['dog_stdv']

        # classify
        if abs(cat_zscore) > abs(dog_zscore):
            if cat_zscore >= 0:
                category = "cat_lover"
            else:
                category = "cat_hater"
        else:
            if dog_zscore >= 0:
                category = "dog_lover"
            else:
                category = "dog_hater"

        response = {
            'category': category,
            'cat_affinity_score': cat_affinity_score,
            'dog_affinity_score': dog_affinity_score,
            'cat_zscore': cat_zscore,
            'cat_zscore': dog_zscore
        }

        transformed_response = self.response_transformer.transform_response(response)
        return transformed_response

## Construct `MyModel`
Inject Model with Request Transformer, Response Transformer, and Model Initializer
```
MyModel(`MyRequestTransformer`, `MyResponseTransformer`, `MyModelInitializer`)
```

In [ ]:
pio_model = MyModel(MyRequestTransformer(),
                MyResponseTransformer(), 
                MyModelInitializer())

## Save/Pickle Model as `pio_model.pkl`

In [ ]:
import dill as pickle

pio_model_pkl_path = 'pio_model.pkl'

with open(pio_model_pkl_path, 'wb') as fh:
    pickle.dump(pio_model, fh)

In [ ]:
import subprocess

output = subprocess.check_output('ls -l model.pkl',
                                 stderr=subprocess.STDOUT,
                                 shell=True)

print(output.decode('utf-8'))

In [ ]:
model_pkl_path='pio_model.pkl'
test_request_path='data/test_request.json'
test_response_path='data/test_response.json'

In [ ]:
from __future__ import print_function, absolute_import, division

import json
import importlib
import dill as pickle

def test(pio_model_pkl_path, test_request_path, test_response_path):
    with open(pio_model_pkl_path, 'rb') as fh:
        pio_model = pickle.load(fh)
    with open(test_request_path, 'rb') as fh:
        actual_request = fh.read() 
    with open(test_response_path, 'rb') as fh:
        expected_response = fh.read()
    print('Expected Response:')
    print(expected_response)
    print('')
    print('Actual Request:')
    print(actual_request)
    actual_response = model.predict(actual_request)
    print('')
    print('Actual Response:')
    print(actual_response)

    return (json.loads(expected_response.decode('utf-8').strip()) \
            == json.loads(actual_response.strip()))

In [ ]:
test_success = test(model_pkl_path, 
                    test_request_path,
                    test_response_path)

print('')
print('Test Success: %s' % test_success)